# Codon Read Through Analysis in Drosophila

This notebook analyzes codon read through in Drosophila melanogaster using transcript and CDS sequences from FlyBase.

## Overview
- Parse transcript and CDS FASTA files
- Associate CDS with parent transcripts
- Extract 5' UTR, ORF1, and 3' UTR regions
- Analyze amino acid sequences, stop codons, and potential ORF2
- Compare with externally validated readthrough candidates

In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re
import warnings
from collections import defaultdict

warnings.filterwarnings('ignore')

## Step 1: Parse FASTA files and associate CDS with transcripts

In [ ]:
def parse_fasta_header(header):
    """Parse FASTA header to extract metadata."""
    metadata = {}
    
    # Extract ID (first part of header)
    parts = header.split(' ', 1)
    metadata['id'] = parts[0]
    
    if len(parts) > 1:
        # Parse key=value pairs
        attributes = parts[1]
        
        # Handle parent field which can have multiple values
        parent_match = re.search(r'parent=([^;]+)', attributes)
        if parent_match:
            metadata['parent'] = parent_match.group(1).split(',')
        
        # Extract other fields
        for field in ['type', 'name', 'length']:
            match = re.search(f'{field}=([^;]+)', attributes)
            if match:
                metadata[field] = match.group(1)
    
    return metadata

def load_sequences(fasta_file):
    """Load sequences from FASTA file with parsed headers."""
    sequences = {}
    
    for record in SeqIO.parse(fasta_file, 'fasta'):
        metadata = parse_fasta_header(record.description)
        sequences[metadata['id']] = {
            'sequence': str(record.seq),
            'metadata': metadata
        }
    
    return sequences

print("Loading transcript sequences...")
transcripts = load_sequences('dmel-all-transcript-r6.64.fasta')
print(f"Loaded {len(transcripts)} transcripts")

print("Loading CDS sequences...")
cds_sequences = load_sequences('dmel-all-CDS-r6.64.fasta')
print(f"Loaded {len(cds_sequences)} CDS sequences")

In [ ]:
# Show example of parsed data
transcript_id = list(transcripts.keys())[0]
print(f"Example transcript: {transcript_id}")
print(f"Metadata: {transcripts[transcript_id]['metadata']}")
print(f"Sequence length: {len(transcripts[transcript_id]['sequence'])}")
print()

cds_id = list(cds_sequences.keys())[0]
print(f"Example CDS: {cds_id}")
print(f"Metadata: {cds_sequences[cds_id]['metadata']}")
print(f"Sequence length: {len(cds_sequences[cds_id]['sequence'])}")

In [ ]:
def associate_cds_transcripts(transcripts, cds_sequences):
    """Associate each CDS with its parent transcript."""
    associations = {}
    warnings_count = 0
    
    for cds_id, cds_data in cds_sequences.items():
        if 'parent' not in cds_data['metadata']:
            print(f"Warning: CDS {cds_id} has no parent information")
            warnings_count += 1
            continue
            
        parents = cds_data['metadata']['parent']
        
        # Find transcript parent (starts with FBtr)
        transcript_parent = None
        for parent in parents:
            if parent.startswith('FBtr'):
                transcript_parent = parent
                break
        
        if not transcript_parent:
            print(f"Warning: CDS {cds_id} has no transcript parent")
            warnings_count += 1
            continue
            
        if transcript_parent not in transcripts:
            print(f"Warning: Transcript {transcript_parent} not found for CDS {cds_id}")
            warnings_count += 1
            continue
        
        # Check for one-to-one mapping
        if transcript_parent in associations:
            print(f"Warning: Transcript {transcript_parent} already associated with another CDS")
            warnings_count += 1
            continue
            
        associations[transcript_parent] = cds_id
    
    print(f"Successfully associated {len(associations)} CDS-transcript pairs")
    print(f"Issued {warnings_count} warnings for problematic associations")
    
    return associations

cds_transcript_map = associate_cds_transcripts(transcripts, cds_sequences)

In [ ]:
def align_cds_to_transcript(transcript_seq, cds_seq):
    """Align CDS to transcript and extract UTRs and ORF."""
    # Find CDS in transcript
    cds_start = transcript_seq.find(cds_seq)
    
    if cds_start == -1:
        return None
    
    cds_end = cds_start + len(cds_seq)
    
    # Extract regions
    utr5 = transcript_seq[:cds_start]
    orf1 = cds_seq
    utr3 = transcript_seq[cds_end:]
    
    return {
        'utr5': utr5,
        'orf1': orf1,
        'utr3': utr3,
        'cds_start': cds_start,
        'cds_end': cds_end
    }

def create_gene_dataframe(transcripts, cds_sequences, cds_transcript_map):
    """Create dataframe with gene information."""
    data = []
    failed_alignments = 0
    
    for transcript_id, cds_id in cds_transcript_map.items():
        transcript_seq = transcripts[transcript_id]['sequence']
        cds_seq = cds_sequences[cds_id]['sequence']
        
        # Get gene ID from transcript metadata
        gene_id = None
        if 'parent' in transcripts[transcript_id]['metadata']:
            for parent in transcripts[transcript_id]['metadata']['parent']:
                if parent.startswith('FBgn'):
                    gene_id = parent
                    break
        
        if not gene_id:
            gene_id = transcript_id  # fallback
        
        alignment = align_cds_to_transcript(transcript_seq, cds_seq)
        
        if alignment is None:
            failed_alignments += 1
            continue
            
        data.append({
            'gene_id': gene_id,
            'transcript_id': transcript_id,
            'cds_id': cds_id,
            'utr5': alignment['utr5'],
            'orf1': alignment['orf1'],
            'utr3': alignment['utr3']
        })
    
    print(f"Successfully processed {len(data)} genes")
    print(f"Failed to align {failed_alignments} CDS sequences")
    
    return pd.DataFrame(data)

# Create the initial dataframe
df = create_gene_dataframe(transcripts, cds_sequences, cds_transcript_map)
print(f"Created dataframe with {len(df)} rows")
df.head()

## Step 2: Add sequence analysis columns

In [ ]:
def translate_sequence(seq):
    """Translate nucleotide sequence to amino acids."""
    if len(seq) % 3 != 0:
        seq = seq[:-(len(seq) % 3)]  # trim to multiple of 3
    
    bio_seq = Seq(seq)
    return str(bio_seq.translate())

def find_stop_codons(seq):
    """Find all stop codons in sequence."""
    stop_codons = ['TAA', 'TAG', 'TGA']
    positions = []
    
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        if codon in stop_codons:
            positions.append((i, codon))
    
    return positions

def analyze_sequences(df):
    """Add sequence analysis columns to dataframe."""
    
    # Initialize new columns
    df['aa_sequence_orf1'] = ''
    df['stop_codon_orf1'] = ''
    df['bp_after_stop'] = ''
    df['utr3_length'] = 0
    df['first_stop_after_canonical'] = ''
    df['orf2_translation'] = ''
    df['orf1_nucleotide'] = ''
    df['orf2_nucleotide'] = ''
    
    for idx, row in df.iterrows():
        orf1 = row['orf1']
        utr3 = row['utr3']
        
        # AA sequence of ORF1
        df.at[idx, 'aa_sequence_orf1'] = translate_sequence(orf1)
        
        # Stop codon at end of ORF1
        if len(orf1) >= 3:
            df.at[idx, 'stop_codon_orf1'] = orf1[-3:]
        
        # Base pairs following the stop (start of 3' UTR)
        df.at[idx, 'bp_after_stop'] = utr3[:10] if len(utr3) >= 10 else utr3
        
        # Length of 3' UTR
        df.at[idx, 'utr3_length'] = len(utr3)
        
        # Find first stop codon in 3' UTR (potential ORF2 end)
        stop_positions = find_stop_codons(utr3)
        if stop_positions:
            df.at[idx, 'first_stop_after_canonical'] = stop_positions[0][1]
            orf2_end = stop_positions[0][0] + 3
        else:
            df.at[idx, 'first_stop_after_canonical'] = ''
            orf2_end = len(utr3)
        
        # ORF2 sequences (starting immediately after ORF1 stop codon)
        orf2_seq = utr3[:orf2_end]
        df.at[idx, 'orf2_nucleotide'] = orf2_seq
        
        # Translate ORF2
        if orf2_seq:
            df.at[idx, 'orf2_translation'] = translate_sequence(orf2_seq)
        
        # ORF1 nucleotide sequence
        df.at[idx, 'orf1_nucleotide'] = orf1
    
    return df

# Apply sequence analysis
df = analyze_sequences(df)
print("Added sequence analysis columns")
print(f"Dataframe now has {df.shape[1]} columns")

In [ ]:
# Show some examples of the analysis
print("Sample of sequence analysis:")
sample_cols = ['gene_id', 'stop_codon_orf1', 'utr3_length', 'first_stop_after_canonical', 'orf2_translation']
print(df[sample_cols].head())

## Step 3: Add readthrough validation data

In [ ]:
def load_readthrough_candidates(filename):
    """Load externally validated readthrough candidates."""
    readthrough_genes = set()
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    # Extract gene ID (assuming it's the first column or contains FBgn)
                    parts = line.split('\t')
                    for part in parts:
                        if 'FBgn' in part:
                            readthrough_genes.add(part.strip())
                            break
                    else:
                        # If no FBgn found, try first column
                        if parts:
                            readthrough_genes.add(parts[0].strip())
    
    except FileNotFoundError:
        print(f"Warning: {filename} not found")
        return set()
    
    return readthrough_genes

# Load readthrough candidates
readthrough_candidates = load_readthrough_candidates('Data1_DmelReadthroughCandidates.txt')
print(f"Loaded {len(readthrough_candidates)} readthrough candidates")

# Show first few candidates
print("First 10 readthrough candidates:")
for i, candidate in enumerate(list(readthrough_candidates)[:10]):
    print(f"  {candidate}")

In [ ]:
# Add readthrough column to dataframe
df['has_readthrough'] = df['gene_id'].isin(readthrough_candidates)

print(f"Genes with validated readthrough: {df['has_readthrough'].sum()}")
print(f"Total genes in analysis: {len(df)}")
print(f"Percentage with readthrough: {df['has_readthrough'].sum() / len(df) * 100:.2f}%")

## Summary and Analysis

In [ ]:
# Final dataframe overview
print("Final dataframe columns:")
for col in df.columns:
    print(f"  {col}")

print(f"\nDataframe shape: {df.shape}")

# Show complete example
print("\nComplete example (first readthrough gene):")
readthrough_example = df[df['has_readthrough']].iloc[0] if df['has_readthrough'].any() else df.iloc[0]
for col, val in readthrough_example.items():
    if isinstance(val, str) and len(val) > 50:
        print(f"{col}: {val[:50]}...")
    else:
        print(f"{col}: {val}")

In [ ]:
# Basic analysis of readthrough genes
print("Analysis of genes with readthrough:")
readthrough_df = df[df['has_readthrough']]

if len(readthrough_df) > 0:
    print(f"Average 3' UTR length in readthrough genes: {readthrough_df['utr3_length'].mean():.1f} bp")
    print(f"Average 3' UTR length in non-readthrough genes: {df[~df['has_readthrough']]['utr3_length'].mean():.1f} bp")
    
    print(f"\nStop codons in readthrough genes:")
    print(readthrough_df['stop_codon_orf1'].value_counts())
    
    print(f"\nStop codons in non-readthrough genes:")
    print(df[~df['has_readthrough']]['stop_codon_orf1'].value_counts())
    
    # Genes with potential ORF2 (have stop codon in 3' UTR)
    has_orf2 = readthrough_df['first_stop_after_canonical'] != ''
    print(f"\nReadthrough genes with potential ORF2: {has_orf2.sum()} / {len(readthrough_df)}")
else:
    print("No readthrough genes found in the analysis!")

In [ ]:
# Save the final dataframe
df.to_csv('drosophila_readthrough_analysis.csv', index=False)
print("Saved analysis results to 'drosophila_readthrough_analysis.csv'")

# Save summary statistics
with open('analysis_summary.txt', 'w') as f:
    f.write("Drosophila Readthrough Analysis Summary\n")
    f.write("=====================================\n\n")
    f.write(f"Total genes analyzed: {len(df)}\n")
    f.write(f"Genes with validated readthrough: {df['has_readthrough'].sum()}\n")
    f.write(f"Percentage with readthrough: {df['has_readthrough'].sum() / len(df) * 100:.2f}%\n\n")
    
    if df['has_readthrough'].any():
        readthrough_df = df[df['has_readthrough']]
        f.write(f"Average 3' UTR length (readthrough): {readthrough_df['utr3_length'].mean():.1f} bp\n")
        f.write(f"Average 3' UTR length (non-readthrough): {df[~df['has_readthrough']]['utr3_length'].mean():.1f} bp\n")
        
        has_orf2 = readthrough_df['first_stop_after_canonical'] != ''
        f.write(f"Readthrough genes with potential ORF2: {has_orf2.sum()} / {len(readthrough_df)}\n")

print("Saved summary to 'analysis_summary.txt'")